# Refactoring the feature engineering steps

We forecasted 24 hours of pollutant concentration using a naive forecast and then a simple linear regression with the previous hr's pollutant concentration as an input variable.

In Section 2, we saw that we can actually extract a lot of potentially valuable features both from the timeseries and its accompanying timestamp. We created most of the features by utilizing pandas. This way of engineering features has some limitations. For example, we don't leave behind code to create features from new input data. In fact, we would have to re-write the entire block of code for every new input data set. This is not efficient.

To avoid this, we can create classes with fit() and transform() functionality that can be easily deployed on new input data to recreate the features given the new information.

In this notebook, we will modify the code to make it more suitable for multi-step forecasting.

We will package the feature engineering code into Scikit-learn-like classes or replace pandas multi-line code with transformers from the open source library Feature-engine, which works like Scikit-learn.

For simplicity, we will only predict the concentration of CO.

**If you haven't done so already, check the Air Pollutants notebook in Section 2 to familiarize yourself with the feature engineering steps we used previously.**


## Data

We will work with the Air Quality Dataset from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Air+Quality).

For instructions on how to download, prepare, and store the dataset, refer to notebook number 3, in the folder "01-Datasets" from this repo.

In [1]:
import numpy as np
import pandas as pd

# for our new classes
from sklearn.base import BaseEstimator, TransformerMixin

# to automate many of our engineering processes
from feature_engine.creation import CyclicalTransformer, MathematicalCombination
from feature_engine.datetime import DatetimeFeatures
from feature_engine.imputation import MeanMedianImputer

# Load Data

In [2]:
# Function to load and prepare input data.

def load_data():
    
    # Data lives here.
    filename = '../datasets/AirQualityUCI_ready.csv'

    # Load data: only the time variable and CO
    data = pd.read_csv(filename, usecols=['Date_Time', 'CO_sensor'])
        
    # Cast date variable in datetime format.
    data['Date_Time'] = pd.to_datetime(data['Date_Time'])

    # Set the index to the timestamp.
    data.index = data['Date_Time']

    # Sanity: sort index.
    data.sort_index(inplace=True)
    
    # Reduce data span.
    data = data[(
        data['Date_Time'] >= '2004-04-01') &
        (data['Date_Time'] <= '2005-04-30')
    ]
    
    # Remove outliers
    data = data.loc[(data['CO_sensor']>0)]
    
    return data

In [3]:
# Load data.

data = load_data()

data.head()

,Date_Time,CO_sensor
Date_Time,,
2004-04-04 00:00:00,2004-04-04 00:00:00,1224.0
2004-04-04 01:00:00,2004-04-04 01:00:00,1215.0
2004-04-04 02:00:00,2004-04-04 02:00:00,1115.0
2004-04-04 03:00:00,2004-04-04 03:00:00,1124.0
2004-04-04 04:00:00,2004-04-04 04:00:00,1028.0


# Feature engineering

Throughout this notebook, we will create multiple classes or use open source to create new features from the time series and its timestamp.

## Extract time related features

We can extract date and time features automatically utilizing Feature-engine.

[DatetimeFeatures](add url)

In [4]:
dt_features = DatetimeFeatures(
    
    # the timestamp
    variables='Date_Time',
    
    # the features we want from the timestamp
    features_to_extract=["month",
                         "week_of_the_year",
                         "day_of_the_week",
                         "day_of_the_month",
                         "hour",
                         "weekend",
                        ],
    
    # if we want to drop the timestamp.
    drop_original=True
)

# Extract the datetime features
data = dt_features.fit_transform(data)

# Show new variables
data[[v for v in data.columns if "Date_Time"in v]].head()

,Date_Time_month,Date_Time_woty,Date_Time_dotw,Date_Time_dotm,Date_Time_hour,Date_Time_weekend
Date_Time,,,,,,
2004-04-04 00:00:00,4,14,6,4,0,1
2004-04-04 01:00:00,4,14,6,4,1,1
2004-04-04 02:00:00,4,14,6,4,2,1
2004-04-04 03:00:00,4,14,6,4,3,1
2004-04-04 04:00:00,4,14,6,4,4,1


## Lag Features

We create the following lagged features:

- The pollutant concentration for the previous hour (t-1).

- The pollutant concentration for the same hour on the previous day (t-24).

In [5]:
class LagFeatures(BaseEstimator, TransformerMixin):

    def __init__(self, features, frequency, label):

        # In the init we specify the parameters that
        # the user needs to pass to start the transformer.

        # The user needs to indicate which features to lag,
        # how much we should lag the variables, 
        # and the name for the new variables.
        
        self.features = features
        self.frequency = frequency
        self.label = label

    def fit(self, X, y=None):

        # We do not need to learn parameters

        return self

    def transform(self, X):

        # We lag the features

        # We make a copy not to over-write the original data
        X = X.copy()

        # Shift the data forward.
        tmp = X[self.features].shift(freq=self.frequency)

        # Name the new variables.
        tmp.columns = [v + self.label for v in self.features]

        # Add the variables to the original data.
        X = X.merge(tmp, left_index=True, right_index=True, how='left')

        return X

In [6]:
# Add the lag 1 Hr features.

lag1 = LagFeatures(['CO_sensor'], '1H', '_lag_1')

data = lag1.fit_transform(data)

data[['CO_sensor', 'CO_sensor_lag_1']].head()

,CO_sensor,CO_sensor_lag_1
Date_Time,,
2004-04-04 00:00:00,1224.0,NaN
2004-04-04 01:00:00,1215.0,1224.0
2004-04-04 02:00:00,1115.0,1215.0
2004-04-04 03:00:00,1124.0,1115.0
2004-04-04 04:00:00,1028.0,1124.0


We see for example that 1224 is now moved forward to the next t.

In [7]:
# Add the lag 24 Hr features.

lag24 = LagFeatures(['CO_sensor'], '24H', '_lag_24')

data = lag24.fit_transform(data)

data[['CO_sensor', 'CO_sensor_lag_24']].head(25)

,CO_sensor,CO_sensor_lag_24
Date_Time,,
2004-04-04 00:00:00,1224.0,NaN
2004-04-04 01:00:00,1215.0,NaN
2004-04-04 02:00:00,1115.0,NaN
2004-04-04 03:00:00,1124.0,NaN
2004-04-04 04:00:00,1028.0,NaN
2004-04-04 05:00:00,1010.0,NaN
2004-04-04 06:00:00,1074.0,NaN
2004-04-04 07:00:00,1034.0,NaN
2004-04-04 08:00:00,1130.0,NaN


Note now that 1224, which is the value corresponding to April 4 at midnight, is now located on April 5th at midnight. We have NA for all previous values because there is no information about the features 24 hours before.

## Window features

We take the average of the previous 3 hours of the time series to predict the current hour. 

We first need to calculate the average of the 3 previous values, and then move that value forward.

In [8]:
class WindowFeatures(BaseEstimator, TransformerMixin):

    def __init__(self, features, window, frequency):

        # In the init we specify the parameters that
        # the user needs to pass to start the transformer.

        # The user needs to indicate the features to use for the computation
        # the size of the window,
        # and the frequency to shift forward.
        
        self.features = features
        self.window = window
        self.frequency = frequency

    def fit(self, X, y=None):

        # We do not need to learn parameters

        return self

    def transform(self, X):

        # First we calculate the average of the feature in
        # the indicated window, then we shift the value forward
        # based on the indicated frequency.

        X = X.copy()

        tmp = (X[self.features]
               .rolling(window=self.window).mean()
               .shift(freq=self.frequency)
               )

        # Rename the columns
        tmp.columns = [v + '_window' for v in self.features]

        # Add the variables to the original data.
        X = X.merge(tmp, left_index=True, right_index=True, how='left')

        return X

In [9]:
# With the new class, we add the window features

win_feat = WindowFeatures(['CO_sensor'], '3H', '1H')

data = win_feat.fit_transform(data)

data[['CO_sensor', 'CO_sensor_window']].head()

,CO_sensor,CO_sensor_window
Date_Time,,
2004-04-04 00:00:00,1224.0,NaN
2004-04-04 01:00:00,1215.0,1224.000000
2004-04-04 02:00:00,1115.0,1219.500000
2004-04-04 03:00:00,1124.0,1184.666667
2004-04-04 04:00:00,1028.0,1151.333333


**Important:** Notice how the average of the previous three hours was advanced an hour to time t, the time we want to forecast.

## Feature combination.

To simply show how we can combine features, we will determine the mean value of the pollutant at times t-1 and t-24. I am not sure if it adds a huge value, but the idea is to show you an extra feature engineering step. So indulge me for a minute ;)

We will automate this process with Feature-engine.

[MathematicalCombination](add url)

In [10]:
combine = MathematicalCombination(
    
    # the variables to average
    variables_to_combine=['CO_sensor_lag_1','CO_sensor_lag_24'],
    
    # we indicate we want the average
    math_operations=['mean'],
    
    # the name of the new feature
    new_variables_names=['CO_lag_ave'],
    
    # what to do if the variables have NA
    missing_values='ignore',
)

data = combine.fit_transform(data)

data.head()

,CO_sensor,Date_Time_month,Date_Time_woty,Date_Time_dotw,Date_Time_dotm,Date_Time_hour,Date_Time_weekend,CO_sensor_lag_1,CO_sensor_lag_24,CO_sensor_window,CO_lag_ave
Date_Time,,,,,,,,,,,
2004-04-04 00:00:00,1224.0,4,14,6,4,0,1,NaN,NaN,NaN,NaN
2004-04-04 01:00:00,1215.0,4,14,6,4,1,1,1224.0,NaN,1224.000000,1224.0
2004-04-04 02:00:00,1115.0,4,14,6,4,2,1,1215.0,NaN,1219.500000,1215.0
2004-04-04 03:00:00,1124.0,4,14,6,4,3,1,1115.0,NaN,1184.666667,1115.0
2004-04-04 04:00:00,1028.0,4,14,6,4,4,1,1124.0,NaN,1151.333333,1124.0


## Periodic Features

We transform the month and the hour with the sine and cosine to have a periodic representation of the features.

We automate this procedure with Feature-engine.

[CyclicalTransformer](https://feature-engine.readthedocs.io/en/latest/creation/CyclicalTransformer.html)

In [11]:
# Create features that capture the cyclical representation.

cyclical = CyclicalTransformer(
    
    # The features we want to transform.
    variables=['Date_Time_month', 'Date_Time_hour'],
    
    # Whether to drop the original features.
    drop_original=False, 
)

data = cyclical.fit_transform(data)

data.head()

,CO_sensor,Date_Time_month,Date_Time_woty,Date_Time_dotw,Date_Time_dotm,Date_Time_hour,Date_Time_weekend,CO_sensor_lag_1,CO_sensor_lag_24,CO_sensor_window,CO_lag_ave,Date_Time_month_sin,Date_Time_month_cos,Date_Time_hour_sin,Date_Time_hour_cos
Date_Time,,,,,,,,,,,,,,,
2004-04-04 00:00:00,1224.0,4,14,6,4,0,1,NaN,NaN,NaN,NaN,0.866025,-0.5,0.000000,1.000000
2004-04-04 01:00:00,1215.0,4,14,6,4,1,1,1224.0,NaN,1224.000000,1224.0,0.866025,-0.5,0.269797,0.962917
2004-04-04 02:00:00,1115.0,4,14,6,4,2,1,1215.0,NaN,1219.500000,1215.0,0.866025,-0.5,0.519584,0.854419
2004-04-04 03:00:00,1124.0,4,14,6,4,3,1,1115.0,NaN,1184.666667,1115.0,0.866025,-0.5,0.730836,0.682553
2004-04-04 04:00:00,1028.0,4,14,6,4,4,1,1124.0,NaN,1151.333333,1124.0,0.866025,-0.5,0.887885,0.460065


We can see the newly created features at the end of the dataframe.

## Impute Missing data

When creating lag and window features, we introduced some missing data.

In [12]:
data.isnull().sum()

CO_sensor                0
Date_Time_month          0
Date_Time_woty           0
Date_Time_dotw           0
Date_Time_dotm           0
Date_Time_hour           0
Date_Time_weekend        0
CO_sensor_lag_1         27
CO_sensor_lag_24       461
CO_sensor_window        27
CO_lag_ave              17
Date_Time_month_sin      0
Date_Time_month_cos      0
Date_Time_hour_sin       0
Date_Time_hour_cos       0
dtype: int64

We will replace NA by the mean value of the variable.

In [13]:
# We start the imputer with the variables
# we want to impute

imputer = MeanMedianImputer(
    # the variables to impute
    variables = [
        'CO_sensor_lag_1',
        'CO_sensor_lag_24',
        'CO_sensor_window',
        'CO_lag_ave',
    ],
)

# with fit() the transformer learns the mean value
# per variable.

# With transform() it replaces NA by the mean.
data = imputer.fit_transform(data)

In [14]:
# the mean value per variable

imputer.imputer_dict_

{'CO_sensor_lag_1': 1050.0,
 'CO_sensor_lag_24': 1053.0,
 'CO_sensor_window': 1054.0,
 'CO_lag_ave': 1062.0}

## Seasonality Features

We want to calculate the mean pollutant concentration per hour. 

This is the first class that we create where we need to learn parameters from the data.

In [15]:
class SeasonalTransformer(BaseEstimator, TransformerMixin):

    def __init__(self, season_var, variables):

        # In the init we specify the parameters that
        # the user needs to pass to start the transformer.

        # The user needs to indicate the seasonal variable
        # and the variables that should be aggregated.

        self.season_var = season_var
        self.variables = variables

    def fit(self, X, y=None):

        # We want to estimate the mean value of the
        # time series in the seasonal term.

        # In our demo, that is the mean pollutant's 
        # concentration per hour.

        # We make a copy of the dataframe 
        # not to over-write the user's data.
        X = X.copy()

        # Calculate mean pollutant per hr.
        # The learned values will be stored in this attribute.
        self.seasonal_ = X.groupby(self.season_var)[self.variables].mean()

        # Rename the new variables.
        self.seasonal_.columns = [v + '_season' for v in self.variables]
        
        # Ensure returned grouping is a dataframe
        self.seasonal_ = self.seasonal_.reset_index()

        return self

    def transform(self, X):

        # Add the seasonal component to the
        # dataset to transform.

        X = X.copy()
        
        # Store the datetime index (it is lost in merge)
        index = X.index

        # Add the seasonal feature
        X = X.merge(self.seasonal_, on=self.season_var, how='left')
        
        # restore the datetime index to the df
        X.index = index

        return X

In [16]:
seasonal_t = SeasonalTransformer(
    
    # the variable for the grouping
    season_var='Date_Time_hour', 
    
    # the variables to group
    variables=['CO_sensor'],
)

# Add the seasonal features
data = seasonal_t.fit_transform(data)

data[['Date_Time_hour', 'CO_sensor_season']].head()

,Date_Time_hour,CO_sensor_season
Date_Time,,
2004-04-04 00:00:00,0,1051.169935
2004-04-04 01:00:00,1,990.540717
2004-04-04 02:00:00,2,930.584416
2004-04-04 03:00:00,3,892.711974
2004-04-04 04:00:00,4,874.703226


In [17]:
# The transformer learned and stored the mean
# pollutant concentration per hour.

seasonal_t.seasonal_.head()

,Date_Time_hour,CO_sensor_season
0,0,1051.169935
1,1,990.540717
2,2,930.584416
3,3,892.711974
4,4,874.703226


In the two views of the dataframes above, we can see how the seasonal features with the mean pollutant concentration per hour were added to the corresponding hours in the main data set.

## Wrapping up

We have now created a lot of features that we can use to predict the pollutant concentration. And we have also created many classes that we can put in a sequence inside a Pipeline.

In the next notebook, we will see how we can integrate the feature engineering steps inside a pipeline, and make a single point forecast.

That is all for this notebook. I hope you enjoyed it!